# LIMIT-GRAPH v1.1: Multilingual Extension Demo

This notebook demonstrates the multilingual capabilities of LIMIT-GRAPH v1.1, including:
- Cross-lingual evaluation of retrieval and memory agents
- Semantic graph structure preservation across languages
- Entity linking and graph traversal in multilingual corpora

## Supported Languages
- Indonesian (id)
- Spanish (es)
- English (en)

## 1. Setup - Load Multilingual Corpus and Graph

In [ ]:
import json
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from typing import Dict, List
import sys
import os

# Add the agents directory to the path
sys.path.append('agents')
from multilingual_entity_linker import MultilingualEntityLinker, EntityMatch, GraphTraversalPath

# Language selection - Change this to test different languages
LANGUAGE = 'id'  # Options: 'id', 'es', 'en'

print(f"Selected language: {LANGUAGE}")
print(f"Loading {LANGUAGE} dataset...")

In [ ]:
def load_jsonl(filepath):
    """Load JSONL file"""
    data = []
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                data.append(json.loads(line.strip()))
    except FileNotFoundError:
        print(f"Warning: {filepath} not found")
    return data

# Load multilingual datasets
corpus = load_jsonl(f'data/corpus_{LANGUAGE}.jsonl')
graph_edges = load_jsonl(f'data/graph_edges_{LANGUAGE}.jsonl')
queries = load_jsonl(f'data/queries_{LANGUAGE}.jsonl')

print(f"Loaded {len(corpus)} documents")
print(f"Loaded {len(graph_edges)} graph edges")
print(f"Loaded {len(queries)} queries")

# Display sample data
print("\nSample corpus:")
for doc in corpus[:2]:
    print(f"  {doc['_id']}: {doc['text']}")

print("\nSample graph edges:")
for edge in graph_edges[:3]:
    print(f"  {edge['source']} --[{edge['relation']}]--> {edge['target']}")

## 2. Entity Linking - Run MultilingualEntityLinker

In [ ]:
# Initialize the multilingual entity linker
entity_linker = MultilingualEntityLinker(graph_edges, lang=LANGUAGE)

# Process sample query
sample_query = queries[0]['text'] if queries else "Sample query"
print(f"Processing query: {sample_query}")

# Run complete query processing
result = entity_linker.process_query(sample_query)

print("\n=== Entity Linking Results ===")
print(f"Extracted entities: {result['extracted_entities']}")
print("\nLinked entities:")
for entity in result['linked_entities']:
    print(f"  '{entity['entity']}' -> '{entity['graph_node']}' (confidence: {entity['confidence']:.2f})")
    if entity['wikidata_qid']:
        print(f"    Wikidata QID: {entity['wikidata_qid']}")

## 3. Graph Traversal - Visualize Reasoning Paths

In [ ]:
# Visualize the semantic graph
plt.figure(figsize=(12, 8))

# Create NetworkX graph for visualization
G = nx.DiGraph()
for edge in graph_edges:
    G.add_edge(edge['source'], edge['target'], relation=edge['relation'])

# Layout and draw
pos = nx.spring_layout(G, k=2, iterations=50)
nx.draw(G, pos, with_labels=True, node_color='lightblue', 
        node_size=2000, font_size=10, font_weight='bold',
        arrows=True, arrowsize=20, edge_color='gray')

# Add edge labels
edge_labels = {(edge['source'], edge['target']): edge['relation'] 
               for edge in graph_edges}
nx.draw_networkx_edge_labels(G, pos, edge_labels, font_size=8)

plt.title(f"Semantic Graph - {LANGUAGE.upper()}")
plt.axis('off')
plt.tight_layout()
plt.show()

# Display traversal paths
print("\n=== Graph Traversal Paths ===")
for i, path in enumerate(result['traversal_paths'][:3]):
    print(f"Path {i+1}: {' -> '.join(path['path'])}")
    print(f"  Relations: {' -> '.join(path['relations'])}")
    print(f"  Confidence: {path['confidence']:.3f}")
    print()

## 4. Retrieval Fusion - Sparse, Dense, and Graph Modules

In [ ]:
# Simulate retrieval fusion results
def simulate_retrieval_fusion(query, corpus, graph_paths):
    """Simulate retrieval fusion from sparse, dense, and graph modules"""
    
    # Sparse retrieval (keyword matching)
    sparse_results = []
    query_words = set(query.lower().split())
    for doc in corpus:
        doc_words = set(doc['text'].lower().split())
        overlap = len(query_words.intersection(doc_words))
        if overlap > 0:
            sparse_results.append({
                'doc_id': doc['_id'],
                'text': doc['text'],
                'score': overlap / len(query_words),
                'method': 'sparse'
            })
    
    # Dense retrieval (simulated semantic similarity)
    dense_results = []
    for doc in corpus:
        # Simulate semantic similarity (in practice, use embeddings)
        semantic_score = 0.8 if any(entity in doc['text'] for entity in result['extracted_entities']) else 0.3
        dense_results.append({
            'doc_id': doc['_id'],
            'text': doc['text'],
            'score': semantic_score,
            'method': 'dense'
        })
    
    # Graph retrieval (based on traversal paths)
    graph_results = []
    for path in graph_paths:
        # Find documents containing path entities
        for doc in corpus:
            if any(node in doc['text'] for node in path['path']):
                graph_results.append({
                    'doc_id': doc['_id'],
                    'text': doc['text'],
                    'score': path['confidence'],
                    'method': 'graph',
                    'reasoning_path': ' -> '.join(path['path'])
                })
    
    return sparse_results, dense_results, graph_results

# Run retrieval fusion
sparse_results, dense_results, graph_results = simulate_retrieval_fusion(
    sample_query, corpus, result['traversal_paths']
)

print("=== Retrieval Fusion Results ===")
print(f"\nSparse Retrieval ({len(sparse_results)} results):")
for res in sparse_results:
    print(f"  {res['doc_id']}: {res['text']} (score: {res['score']:.3f})")

print(f"\nDense Retrieval ({len(dense_results)} results):")
for res in sorted(dense_results, key=lambda x: x['score'], reverse=True):
    print(f"  {res['doc_id']}: {res['text']} (score: {res['score']:.3f})")

print(f"\nGraph Retrieval ({len(graph_results)} results):")
for res in graph_results:
    print(f"  {res['doc_id']}: {res['text']} (score: {res['score']:.3f})")
    print(f"    Reasoning path: {res['reasoning_path']}")

# RL Fusion Decision (simulated)
print("\n=== RL Fusion Decision ===")
fusion_weights = {'sparse': 0.3, 'dense': 0.4, 'graph': 0.3}
print(f"Fusion weights: {fusion_weights}")
print("Decision: Prioritize dense retrieval with graph reasoning support")

## 5. Provenance Trace - Memory Lineage and Update History

In [ ]:
import datetime

# Simulate provenance trace
provenance_trace = {
    'query_id': queries[0]['_id'] if queries else 'q1',
    'timestamp': datetime.datetime.now().isoformat(),
    'language': LANGUAGE,
    'processing_steps': [
        {
            'step': 'entity_extraction',
            'timestamp': datetime.datetime.now().isoformat(),
            'input': sample_query,
            'output': result['extracted_entities'],
            'confidence': 0.95
        },
        {
            'step': 'entity_linking',
            'timestamp': datetime.datetime.now().isoformat(),
            'input': result['extracted_entities'],
            'output': [e['graph_node'] for e in result['linked_entities']],
            'confidence': sum(e['confidence'] for e in result['linked_entities']) / len(result['linked_entities']) if result['linked_entities'] else 0
        },
        {
            'step': 'graph_traversal',
            'timestamp': datetime.datetime.now().isoformat(),
            'input': [e['graph_node'] for e in result['linked_entities']],
            'output': [p['path'] for p in result['traversal_paths'][:3]],
            'confidence': sum(p['confidence'] for p in result['traversal_paths'][:3]) / min(3, len(result['traversal_paths'])) if result['traversal_paths'] else 0
        }
    ],
    'memory_updates': [
        {
            'type': 'entity_cache',
            'entities_added': result['extracted_entities'],
            'timestamp': datetime.datetime.now().isoformat()
        },
        {
            'type': 'graph_cache',
            'paths_cached': len(result['traversal_paths']),
            'timestamp': datetime.datetime.now().isoformat()
        }
    ]
}

print("=== Provenance Trace ===")
print(f"Query ID: {provenance_trace['query_id']}")
print(f"Language: {provenance_trace['language']}")
print(f"Timestamp: {provenance_trace['timestamp']}")

print("\nProcessing Steps:")
for i, step in enumerate(provenance_trace['processing_steps']):
    print(f"  {i+1}. {step['step']}")
    print(f"     Input: {step['input']}")
    print(f"     Output: {step['output']}")
    print(f"     Confidence: {step['confidence']:.3f}")

print("\nMemory Updates:")
for update in provenance_trace['memory_updates']:
    print(f"  - {update['type']}: {update}")

# Save provenance trace
with open(f'provenance_trace_{LANGUAGE}.json', 'w', encoding='utf-8') as f:
    json.dump(provenance_trace, f, indent=2, ensure_ascii=False)
print(f"\nProvenance trace saved to provenance_trace_{LANGUAGE}.json")

## 6. Evaluation - Multilingual Metrics

In [ ]:
def compute_evaluation_metrics(results, graph_stats, provenance_trace):
    """Compute evaluation metrics for multilingual LIMIT-GRAPH"""
    
    # Recall@K (simulated)
    k_values = [1, 3, 5]
    recall_at_k = {}
    
    # Simulate ground truth relevance
    relevant_docs = ['d12', 'd27']  # Assume these are relevant
    retrieved_docs = [res['doc_id'] for res in sparse_results + dense_results]
    
    for k in k_values:
        retrieved_k = retrieved_docs[:k]
        relevant_retrieved = len(set(retrieved_k).intersection(set(relevant_docs)))
        recall_at_k[f'recall@{k}'] = relevant_retrieved / len(relevant_docs) if relevant_docs else 0
    
    # Graph Coverage
    total_nodes = graph_stats['node_count']
    covered_nodes = len(set([e['graph_node'] for e in results['linked_entities']]))
    graph_coverage = covered_nodes / total_nodes if total_nodes > 0 else 0
    
    # Provenance Integrity
    processing_steps = len(provenance_trace['processing_steps'])
    successful_steps = sum(1 for step in provenance_trace['processing_steps'] if step['confidence'] > 0.5)
    provenance_integrity = successful_steps / processing_steps if processing_steps > 0 else 0
    
    # Entity Linking Accuracy
    high_confidence_links = sum(1 for e in results['linked_entities'] if e['confidence'] > 0.8)
    total_links = len(results['linked_entities'])
    linking_accuracy = high_confidence_links / total_links if total_links > 0 else 0
    
    return {
        'language': results['lang'],
        'recall_at_k': recall_at_k,
        'graph_coverage': graph_coverage,
        'provenance_integrity': provenance_integrity,
        'entity_linking_accuracy': linking_accuracy,
        'graph_stats': graph_stats
    }

# Compute metrics
metrics = compute_evaluation_metrics(result, result['graph_stats'], provenance_trace)

print("=== Evaluation Metrics ===")
print(f"Language: {metrics['language']}")
print(f"\nRecall@K:")
for k, score in metrics['recall_at_k'].items():
    print(f"  {k}: {score:.3f}")

print(f"\nGraph Coverage: {metrics['graph_coverage']:.3f}")
print(f"Provenance Integrity: {metrics['provenance_integrity']:.3f}")
print(f"Entity Linking Accuracy: {metrics['entity_linking_accuracy']:.3f}")

print(f"\nGraph Statistics:")
for key, value in metrics['graph_stats'].items():
    print(f"  {key}: {value}")

# Save metrics
with open(f'evaluation_metrics_{LANGUAGE}.json', 'w', encoding='utf-8') as f:
    json.dump(metrics, f, indent=2, ensure_ascii=False)
print(f"\nMetrics saved to evaluation_metrics_{LANGUAGE}.json")

## 7. Cross-Language Comparison

In [ ]:
# Compare across languages (if multiple language files exist)
languages_to_compare = ['id', 'es']
comparison_results = {}

for lang in languages_to_compare:
    try:
        # Load data for each language
        lang_corpus = load_jsonl(f'data/corpus_{lang}.jsonl')
        lang_edges = load_jsonl(f'data/graph_edges_{lang}.jsonl')
        lang_queries = load_jsonl(f'data/queries_{lang}.jsonl')
        
        if lang_corpus and lang_edges and lang_queries:
            # Initialize linker for this language
            lang_linker = MultilingualEntityLinker(lang_edges, lang=lang)
            
            # Process query
            lang_result = lang_linker.process_query(lang_queries[0]['text'])
            
            # Compute basic metrics
            comparison_results[lang] = {
                'entities_extracted': len(lang_result['extracted_entities']),
                'entities_linked': len(lang_result['linked_entities']),
                'traversal_paths': len(lang_result['traversal_paths']),
                'avg_confidence': sum(e['confidence'] for e in lang_result['linked_entities']) / len(lang_result['linked_entities']) if lang_result['linked_entities'] else 0,
                'graph_nodes': lang_result['graph_stats']['node_count'],
                'graph_edges': lang_result['graph_stats']['edge_count']
            }
    except Exception as e:
        print(f"Could not process language {lang}: {e}")

# Display comparison
if comparison_results:
    print("=== Cross-Language Comparison ===")
    comparison_df = pd.DataFrame(comparison_results).T
    print(comparison_df)
    
    # Visualize comparison
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # Entities comparison
    axes[0, 0].bar(comparison_df.index, comparison_df['entities_extracted'])
    axes[0, 0].set_title('Entities Extracted')
    axes[0, 0].set_ylabel('Count')
    
    axes[0, 1].bar(comparison_df.index, comparison_df['entities_linked'])
    axes[0, 1].set_title('Entities Linked')
    axes[0, 1].set_ylabel('Count')
    
    # Confidence comparison
    axes[1, 0].bar(comparison_df.index, comparison_df['avg_confidence'])
    axes[1, 0].set_title('Average Linking Confidence')
    axes[1, 0].set_ylabel('Confidence')
    axes[1, 0].set_ylim(0, 1)
    
    # Graph size comparison
    axes[1, 1].bar(comparison_df.index, comparison_df['graph_nodes'], alpha=0.7, label='Nodes')
    axes[1, 1].bar(comparison_df.index, comparison_df['graph_edges'], alpha=0.7, label='Edges')
    axes[1, 1].set_title('Graph Size')
    axes[1, 1].set_ylabel('Count')
    axes[1, 1].legend()
    
    plt.tight_layout()
    plt.show()
else:
    print("No comparison data available. Make sure multiple language datasets exist.")

## 8. Summary and Next Steps

In [ ]:
print("=== LIMIT-GRAPH v1.1 Demo Summary ===")
print(f"Language processed: {LANGUAGE}")
print(f"Query: {sample_query}")
print(f"Entities extracted: {len(result['extracted_entities'])}")
print(f"Entities linked: {len(result['linked_entities'])}")
print(f"Traversal paths found: {len(result['traversal_paths'])}")
print(f"Graph nodes: {result['graph_stats']['node_count']}")
print(f"Graph edges: {result['graph_stats']['edge_count']}")

print("\n=== Key Features Demonstrated ===")
print("✓ Multilingual entity extraction and linking")
print("✓ Cross-lingual semantic graph preservation")
print("✓ Graph traversal and reasoning path visualization")
print("✓ Retrieval fusion (sparse + dense + graph)")
print("✓ Provenance tracing and memory lineage")
print("✓ Comprehensive evaluation metrics")
print("✓ Cross-language performance comparison")

print("\n=== Next Steps ===")
print("1. Test with different languages by changing LANGUAGE variable")
print("2. Add more complex queries and documents")
print("3. Implement real embedding-based dense retrieval")
print("4. Integrate with Wikidata for better entity alignment")
print("5. Add more sophisticated RL fusion strategies")
print("6. Extend evaluation with human judgments")

print("\n=== Files Generated ===")
print(f"- provenance_trace_{LANGUAGE}.json")
print(f"- evaluation_metrics_{LANGUAGE}.json")
print("\nDemo completed successfully! 🎉")